In [6]:
import json, requests, time, csv, os

In [7]:
startTime = time.time()

In [8]:
USER = 'secret'
PASS = 'secret'
HOST = 'secret'
baseURL = 'secret'

In [9]:
def aspace_auth(host, username, password):
    auth = requests.post(host + '/users/' + username + '/login',
                        params={'password' : password})
    if auth.status_code == 200:
        token = auth.json()['session']
        headers = {'X-ArchivesSpace-Session': token}
        return(headers)
    else:
        return(False)

headers = aspace_auth(HOST, USER, PASS)
print(headers)

{'X-ArchivesSpace-Session': '56e63c58db9e212b8d1c155633fe31b6cfc85ba4b7da46d76c49aa6b944ddfc6'}


In [10]:
# test for successful connection
def test_connection():
    try:
        requests.get(baseURL)
        print('Connected!')
        return True

    except requests.exceptions.ConnectionError:
        print('Connection error. Please confirm ArchivesSpace is running.  Trying again in 10 seconds.')

is_connected = test_connection()

while not is_connected:
    time.sleep(10)
    is_connected = test_connection()

# print instructions
# print 'This script replaces existing fauxcodes with real barcodes (linked in a separate csv file) in ArchivesSpace.'
#raw_input('Press Enter to continue...')

# open csv and generate dict
reader = csv.DictReader(open('rg0051barcodes.csv'))

# GET each top_container listed in top_containers and add to records
print('The following barcodes have been updated in ArchivesSpace:')
for row in reader:
    uri = row['uri']
    output = requests.get(baseURL + uri, headers=headers).json()
    output['barcode'] = row['real']
    post = requests.post(baseURL + uri, headers=headers, data=json.dumps(output)).json()
    print(post)

# show script runtime
elapsedTime = time.time() - startTime
m, s = divmod(elapsedTime, 60)
h, m = divmod(m, 60)
print('Post complete.  Total script run time: ', '%d:%02d:%02d' % (h, m, s))

Connected!
The following barcodes have been updated in ArchivesSpace:


JSONDecodeError: Expecting value: line 1 column 1 (char 0)